In [4]:
#Getting Data
import pandas as pd
from src.extract.nflreadpy_extract import *
from src.transform.transform_module_api import DataTransformerTeam

dfs = []
for year in range(1999, 2026):
    df = get_reg_team_stats(year)
    dfs.append(df)

team_dfs = []
for df in dfs:
    clean = DataTransformerTeam.clean(df)
    team_dfs.append(DataTransformerTeam.team_stats(clean))

years = list(range(1999, 2026))
season_map = {year:df for year,df in zip(years, team_dfs)}

season_map

{1999:    team  pass_yards  rush_yards  pass_attempts  rush_attempts  \
 0   ARI        3085        1207            558            396   
 1   ATL        3691        1196            508            373   
 2   BAL        3172        1694            505            416   
 3   BUF        3478        2040            513            519   
 4   CAR        4447        1527            572            354   
 5   CHI        4352        1387            684            396   
 6   CIN        3504        2051            547            442   
 7   CLE        2997        1150            490            313   
 8   DAL        3278        2051            506            493   
 9   DEN        3646        1864            551            465   
 10  DET        4074        1245            557            356   
 11   GB        4132        1519            604            386   
 12  IND        4182        1660            545            419   
 13  JAX        3716        2091            535            514   
 14 